# Описание проекта

Нужно защитить данные клиентов страховой компании. Разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обосновать корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Описание данных

**Признаки**: 
- пол
- возраст
- зарплата застрахованного
- количество членов его семьи

**Целевой признак**: 
- количество страховых выплат клиенту за последние 5 лет

## Оглавление

1. [Загрузка данных](#step1)
2. [Умножение матриц](#step2)
3. [Алгоритм преобразования](#step3)
4. [Проверка алгоритма](#step4)
5. [Общий вывод](#step5)

<a id="step1"></a>
## 1. Загрузка данных

Подключение библиотек, необходимых модулей и алгоритмов

In [1]:
import pandas as pd
import numpy as np

from IPython.display import display

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

Прочитаем данные из файла. Посмотрим на названия колонок и выведем первые 5 строк.

In [2]:
insurance = pd.read_csv('/datasets/insurance.csv')

print(insurance.columns)
display(insurance.head())

Index(['Пол', 'Возраст', 'Зарплата', 'Члены семьи', 'Страховые выплаты'], dtype='object')


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Можем заметить, что названия столбцов написаны на русском языке, а также в них есть пробелы и заглавные символы. В дальнейшем, чтобы удобнее работать с данными, заменим назавния столбцов на латинские символы.

In [3]:
new_columns = ['gender', 'age', 'salary', 'family', 'payment']

In [4]:
insurance.columns = new_columns
insurance.columns

Index(['gender', 'age', 'salary', 'family', 'payment'], dtype='object')

Выведем общеую информацию по таблице.

In [5]:
insurance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
gender     5000 non-null int64
age        5000 non-null float64
salary     5000 non-null float64
family     5000 non-null int64
payment    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Видим, что тип данных для столбца `age` (возраст) определился как вещественные числа. Краткий обзор данных, дает нам предположение, что эти значения могут быть целочисленные. Посмотрим, на все принимаемые значения этого столбца.

In [6]:
insurance['age'].value_counts()

19.0    223
25.0    214
31.0    212
26.0    211
22.0    209
27.0    209
32.0    206
28.0    204
29.0    203
30.0    202
23.0    202
21.0    200
20.0    195
36.0    193
33.0    191
24.0    182
35.0    179
34.0    177
37.0    147
39.0    141
38.0    139
41.0    129
18.0    117
40.0    114
42.0     93
43.0     77
44.0     74
45.0     73
46.0     60
47.0     47
49.0     37
50.0     27
48.0     26
52.0     22
51.0     21
53.0     11
55.0      9
54.0      7
56.0      5
59.0      3
60.0      2
58.0      2
57.0      2
62.0      1
65.0      1
61.0      1
Name: age, dtype: int64

Действительно, значения целочисленные.

По этой же логике проверим столбец `salary` (зарплата застрахованного).

In [7]:
insurance['salary'].value_counts()

45800.0    29
37100.0    28
43200.0    27
41500.0    27
46800.0    26
           ..
68900.0     1
59700.0     1
59000.0     1
68000.0     1
64400.0     1
Name: salary, Length: 524, dtype: int64

Ситуация такая же. Поменяем типы данных на верные.

In [8]:
insurance['age'] = insurance['age'].astype(int)
insurance['salary'] = insurance['salary'].astype(int)

In [9]:
insurance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
gender     5000 non-null int64
age        5000 non-null int64
salary     5000 non-null int64
family     5000 non-null int64
payment    5000 non-null int64
dtypes: int64(5)
memory usage: 195.4 KB


Стоит отметить, что пропусков нет.

### Вывод

На данном этапе мы выгрузили данные из файла и сохранили в переменной `insurance`. Также провели предобработку данных. Проверили названия столбцов, в них были использованы нелатинские символы, пробелы и заглавные буквы - привели к удобному виду. Подкорректировали автоматически определенные типы данных для столбцов. Проверили, что нет пропущенных значений. В итоге в нашей таблице 5000 строк и 5 столбцов.

<a id="step2"></a>
## 2. Умножение матриц

На данном этапе необходимо определить, изменится ли качество линейной регрессии, если признаки умножить на обратимую матрицу.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

**Формулы линейной регрессии:**

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения в матричном виде:

$$
w = (X^T X)^{-1} X^T y
$$

Для начала выделим отдельно признаки и целевой признак.

In [10]:
features = insurance.drop('payment', axis=1)
target = insurance['payment']

In [11]:
features.shape

(5000, 4)

**Обоснование:**

Посмотрим как будет выглядеть формула обучения (формула нахождения весов), если мы умножим матрицу признаков $X$ на обратимую матрицу $P$. $P$ - квадратная матрица, порядок которой равен количеству признаков.

$ w_1 = ((XP)^T ( XP))^{-1} (X P)^T y $

Проведем преобразования, основываясь на свойствах матричных операций.

$$ (X P)^T (X P) w_1 = (X P)^T y $$

$$ P^T X^T X P w_1 = P^T X^T y $$

Умножим слева на $(P^T)^{-1}$

$$ (P^T)^{-1} P^T X^T X P w_1 = (P^T)^{-1} P^T X^T y $$

Учтем, что $ K K^{-1} = E $, где $K$ - произвольная обратимая матрица, $E$ - единичная матрица.

$$ E X^T X P w_1 = E X^T y $$

$$ X^T X P w_1 = X^T y $$

$$ P w_1 = (X^T X)^{-1} X^T y $$

$$ w_1 = P^{-1} (X^T X)^{-1} X^T y $$

$$ w_1 = P^{-1} w $$

Теперь подставим полученные веса в формулу для нахождения предсказаний:

$$ a_1 = X P w_1 $$

$$ a_1 = X P P^{-1} (X^T X)^{-1} X^T y $$

$$ a_1 = X E (X^T X)^{-1} X^T y $$

$$ a_1 = X (X^T X)^{-1} X^T y $$

$$ a_1 = X w $$

Отсюда следует, что

$$ a_1 = a $$

**Ответ:** Качество линейной регресии не изменится. Выше мы доказали, что если умножить матрицу признаков на обратимую матрицу, то предсказания не изменятся. При этом параметры линейной регресии в преобразованной задаче и в исходной связаны соотношением: $ w_1 = P^{-1} w $ 

### Вывод

На данном шаге мы определили, что качество линейной регрессии не изменится, если умножить признаки на обратимую матрицу.

<a id="step3"></a>
## 3. Алгоритм преобразования

Наша задача разработать метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию, то есть наши признаки. В основе алгоритма преобразования данных учтем тот факт, который мы доказали на предыдущем шаге: если умножить признаки на произвольную обратимую матрицу, то качество линейной регрессии не изменится.

**Алгоритм**

Необходимо сгенерировать квадратную матрицу, где порядок равен количеству признаков. В нашей задаче, количество признаков равно 4. Проверить, что такая матрица обратимая. После чего умножить первоначальные признаки на эту матрицу. Для проверки работы алгоритма, проверить качество линейной регрессии, на этих двух наборах данных.

**Обоснование**

Данный алгоритм обоснован доказательством в шаге 2.

<a id="step4"></a>
## 4. Проверка алгоритма

Для начала получим первоначальную метрику R2 на исходных признаках.

In [12]:
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
r2_orig = r2_score(target, predictions)

print(r2_orig)

0.42494550308169177


Теперь проверим работу нашего алгоритма. Сгенерируем случайную квадратную матрицу с определенным порядком.

In [13]:
size_P = features.shape[1]
P = np.random.normal(size=(size_P, size_P))
display(P)

array([[ 1.04379613, -0.79703997, -0.50415461, -0.08549375],
       [-0.47840073, -1.47003223, -0.45256628,  0.66710262],
       [-0.82657448, -0.1475587 , -0.99843841, -0.00226687],
       [-0.57479398,  0.96075221,  1.00406212,  0.40304857]])

Проверим, что полученная матрица обратимая.

In [14]:
P_inv = np.linalg.inv(P)
display(P_inv)

array([[ 1.39443045, -0.27891847,  0.18506454,  0.75847398],
       [ 1.13289622, -0.58925896,  0.92272823,  1.22080437],
       [-1.32772683,  0.31750357, -1.29406706, -0.81442616],
       [ 2.59571175,  0.21589925,  1.28814383,  2.68158811]])

Видно, что обратная матрица существует. Если бы она была необратима, то возникла бы ошибка. Теперь преобразуем первоначальные признаки и проверим качество линейной регрессии на новых данных.

In [15]:
features_modified = features.dot(P)
features_modified.columns = features.columns

Покажем, что данные изменились от первоначальных.

In [16]:
print('Original features')
display(features.head())
print()
print('Modified features')
display(features_modified.head())

Original features


,gender,age,salary,family
0,1,41,49600,1
1,0,46,38000,1
2,0,29,21000,0
3,0,21,41700,2
4,1,28,26100,0



Modified features


,gender,age,salary,family
0,-41017.239817,-7379.019158,-49540.600590,-84.768056
1,-31432.411606,-5673.891353,-37960.473677,-55.051341
2,-17371.937778,-3141.363647,-20980.331093,-28.258322
3,-34479.351971,-6182.146987,-41642.377586,-79.713282
4,-21585.945447,-3893.240028,-26072.418587,-40.571962


In [17]:
model = LinearRegression()
model.fit(features_modified, target)
predictions_modifiead = model.predict(features_modified)
r2_modified = r2_score(target, predictions_modifiead)

print(r2_modified)

0.424945503081696


Сравним полученные значения метрики R2.

In [18]:
print(abs(r2_modified - r2_orig))

4.218847493575595e-15


Здесь мы получили очень маленькое значение, но не нулевое. Это можно объяснить тем, что в данном случае мы не влиям на точность вещественных чисел, которые используются внутри модели линейной регрессии. Также в качестве обратимой матрицы мы использовали автомитический сгенерированную матрицу с вещественными числами. Однако в данной задаче столь малым различием между полученными значениями можно пренебречь и считать что качества двух моделей, обученной на первоначальных данных и на преобразованных, не отличается.

### Вывод

На данном шаге мы реализовали предложенный алгоритм преобразования данных, чтобы защитить личную информацию. Также на реальных данных было показано, что качество стандартной линейной регресии (из sklearn) не изменилось.

<a id="step5"></a>
## 5. Общий вывод

В данной работе мы научились защищать данные клиентов страховой компании. Разработали алгоритм преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обосновали корректность его работы, а также проверили на реальных данных работу этого алгоритма. Посчитав метрику R2 для первоначальных и преобразованных признаков, показали что качество не изменилось.